# Netflix Movies and TV Shows
- 데이터 셋 https://www.kaggle.com/shivamb/netflix-shows


In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("netflix_titles.csv")

In [ ]:
df.head(1)

**column 설명**
- show_id : 영화 id값(기본키)
- type : movie, tv_show
- title : 영화나 티비 프로그램 제목
- director : 감독
- cast : 배우
- country : 어디서 제작되었는지
- date_added : 넷플릭스에 추가된 날짜
- release_year : 실제로 영화가 제작된 날짜
- rating : 등급/ 나이제한
- duration : 총 시즌
- listed_in : 장르
- description : 프로그램 설명

## null값 처리

In [ ]:
df.director.fillna("No Director", inplace=True)
df.cast.fillna("No Cast", inplace=True)
df.country.fillna("Country Unavailable", inplace=True)
df.dropna(subset=["date_added", "rating"], inplace=True) # null값이 있는 행 drop

In [ ]:
df.info()

### 간단한 인코딩

In [ ]:
df['year_added'] = df['date_added'].apply(lambda x: x.split(" ")[-1])
df['month_added'] = df['date_added'].apply(lambda x: x.split(" ")[0])
df.replace({'TV Show': 0, 'Movie': 1}, df["type"]) # type 인코딩

In [ ]:
ratings_ages = {  # 연령 인코딩
    'TV-PG': 'Older Kids',
    'TV-MA': 'Adults',
    'TV-Y7-FV': 'Older Kids',
    'TV-Y7': 'Older Kids',
    'TV-14': 'Teens',
    'R': 'Adults',
    'TV-Y': 'Kids',
    'NR': 'Adults',
    'PG-13': 'Teens',
    'TV-G': 'Kids',
    'PG': 'Older Kids',
    'G': 'Kids',
    'UR': 'Adults',
    'NC-17': 'Adults'
}

In [ ]:
df['target_ages'] = df['rating'].replace(ratings_ages)

## 시각화

먼저 영화와 tv_show의 분포를 확인합니다.
- 파이 차트로 확인 했을 때 movie가 69.1%로 tv_show의 30.9% 보다 두배가량 높다는 것을 확인할 수 있습니다. 

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Percentation of Netflix Titles that are either Movies or TV Shows')
g = plt.pie(df.type.value_counts(),explode=(0.025,0.025),
            labels=df.type.value_counts().index, colors=['red','green'],
            autopct='%1.1f%%', startangle=180)
plt.show()

어느정도의 차이가 나는지 확인 할 수 있습니다.

In [ ]:
import plotly.graph_objects as go
val = df['type'].value_counts().index
cnt = df['type'].value_counts().values

fig = go.Figure([go.Bar(x=val, y=cnt, marker_color='darkturquoise')])
fig.update_layout(title_text='Netflix Sources Distribution', title_x=0.5)
fig.show()

### 나라별 콘텐츠 수

히스토그램으로 나라별 콘텐츠(영화와 tv쇼를 합한 값)을 보았을 때 미국이 1위, 다음으로 인도 등으로 이루어져 있었습니다.

In [ ]:
countries=pd.crosstab(df["country"],["type"]).sort_values(by="type",ascending=False)
countries.head(10).plot(kind="bar")
plt.legend()
plt.title("COUNTRY WITH HIGHEST NUMBER OF SHOWS")
plt.show()

### 년도별 영화와 tv쇼의 증가 추세확인

-  영화의 실제 개봉년도로 비교했을 때에 2005년쯤 부터 영화 개봉이 tv쇼와 비교해 급격히 늘기 시작했고, 전체 영화와 tv쇼의 합과 추세가 비슷했습니다.
- 그와 비슷하게 넷플릭스 출시 추세도 2015년쯤 부터 급격하게 증가하기 시작했습니다.

In [ ]:
df_movie = df[df['type']=='Movie'].groupby('release_year').count() # 같은 출시년의 영화들의 개수
df_tv = df[df['type']=='TV Show'].groupby('release_year').count()
df_total = df.groupby('release_year').count()

df_movie.reset_index(level=0, inplace=True) # index를 0으로 초기화합니다
df_tv.reset_index(level=0, inplace=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_movie['release_year'], y=df_total['type'],
                         mode='lines',
                         name='total', marker_color='black'))
fig.add_trace(go.Scatter(x=df_movie['release_year'], y=df_movie['type'],
                    mode='lines',
                    name='Movies', marker_color='mediumpurple'))
fig.add_trace(go.Scatter(x=df_tv['release_year'], y=df_tv['type'],
                    mode='lines',
                    name='TV Shows', marker_color='lightcoral'))
fig.update_layout(title_text='Trend Movies vs TV Shows in recent years', title_x=0.5)
fig.show()

In [ ]:
df_movie = df[df['type']=='Movie'].groupby('year_added').count()
df_tv = df[df['type']=='TV Show'].groupby('year_added').count()
df_total = df.groupby('year_added').count()

df_movie.reset_index(level=0, inplace=True)
df_tv.reset_index(level=0, inplace=True)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_movie['year_added'], y=df_total['type'],
                    mode='lines',
                    name='total', marker_color='black'))
fig.add_trace(go.Scatter(x=df_movie['year_added'], y=df_movie['type'],
                    mode='lines',
                    name='Movies', marker_color='mediumpurple'))
fig.add_trace(go.Scatter(x=df_tv['year_added'], y=df_tv['type'],
                    mode='lines',
                    name='TV Shows', marker_color='lightcoral'))
fig.update_layout(title_text='Trend Movies vs TV Shows in year added', title_x=0.5)
fig.show()